In [ ]:
from dataclasses import dataclass, field
from typing import Any, Tuple, List, Dict, Iterable, Callable, Union

import numpy
import h5py

In [ ]:
@dataclass(order=True)
class _BaseData:
    # mappable member for when composited into SeriesData; sorted by
    key: float = field(repr=True, init=False, compare=True)
        
    # initialization arguments; removed after initialization
    file: h5py.File = field(repr=False, compare=False)
    form: str = field(repr=False, compare=False)
    code: str = field(repr=False, compare=False)

    def __post_init__(self) -> None:
        supported: Dict[str, List[str]] = {'flash' : ['plt', 'chk']}
        
        # check for supported codes and formats
        if self.code not in supported:
            raise Exception(f'Code {self.code} is not supported; only code = {[*supported]}')
            
        if self.form not in supported[self.code]:
            raise Exception(f'File format {self.form} is not supported; only form = {supported[self.code]}')

        # process the file based on options
        self.__init_process__()         

        # verify properly initialized object
        if not hasattr(self, 'key'):
            raise NotImplementedError(f'{type(self)} does not initialize a key_float member')      
            
        # remove h5file and option members
        delattr(self, 'file')
        delattr(self, 'form')
        delattr(self, 'code')
        
    #@abstractmethod   
    def __init_process__(self) -> None:
        raise NotImplementedError(f'{type(self)} does not implement an __init_process__ method')     
    
@dataclass
class GeometryData(_BaseData):
    blk_num: int = field(repr=False, init=False, compare=False)
    blk_num_x: int = field(repr=True, init=False, compare=False)
    blk_num_y: int = field(repr=True, init=False, compare=False)
    blk_num_z: int = field(repr=True, init=False, compare=False)
    blk_size_x: int = field(repr=True, init=False, compare=False)
    blk_size_y: int = field(repr=True, init=False, compare=False)
    blk_size_z: int = field(repr=True, init=False, compare=False)
    blk_coords: numpy.ndarray = field(repr=False, init=False, compare=False)
    blk_dict_x: Dict[float, int] = field(repr=True, init=False, compare=False)
    blk_dict_y: Dict[float, int] = field(repr=True, init=False, compare=False)
    blk_dict_z: Dict[float, int] = field(repr=True, init=False, compare=False)

    grd_type: str = field(repr=True, init=False, compare=False)
    grd_dim: int = field(repr=True, init=False, compare=False)
    grd_size: int = field(repr=False, init=False, compare=False)
    grd_size_x: int = field(repr=False, init=False, compare=False)
    grd_size_y: int = field(repr=False, init=False, compare=False)
    grd_size_z: int = field(repr=False, init=False, compare=False)
        
    def __init_process__(self) -> None:
        # pull relavent data from hdf5 file object
        sim_info: List[Tuple[int, bytes]] = list(self.file['sim info'])
        coordinates: numpy.ndarray = self.file['coordinates']
        int_runtime: List[Tuple[bytes, int]] = list(self.file['integer runtime parameters'])
        int_scalars: List[Tuple[bytes, int]] = list(self.file['integer scalars'])
        real_scalars: List[Tuple[bytes, float]] = list(self.file['real scalars'])
            
        # initialize mappable keys
        self.key = float(_first_true(real_scalars, lambda l: 'time' in str(l[0]))[1])

        # initialize grid type
        setup_call: str = _first_true(sim_info, lambda l: l[0] == 9)[1].decode('utf-8')
        if setup_call.find('+ug') != -1:
            self.grd_type = 'uniform'
        elif setup_call.find('+pm4dev') != -1:
            self.grd_type = 'paramesh'
        else:
            raise Exception(f'Unable to determine grid type from sim info field')

        # initialize grid dimensionality
        self.grd_dim = _first_true(int_scalars, lambda l: 'dimensionality' in str(l[0]))[1]

        # initialize coordinates of blocks
        self.blk_coords = numpy.ndarray(coordinates.shape, dtype=numpy.dtype(float))
        self.blk_coords[:, :] = coordinates

        # initialize block data
        if self.grd_type == 'uniform':
            self.blk_num = _first_true(int_scalars, lambda l: 'globalnumblocks' in str(l[0]))[1] 
            self.blk_num_x = _first_true(int_runtime, lambda l: 'iprocs' in str(l[0]))[1]
            self.blk_num_y = _first_true(int_runtime, lambda l: 'jprocs' in str(l[0]))[1]
            self.blk_num_z = 1
            self.blk_size_x = _first_true(int_scalars, lambda l: 'nxb' in str(l[0]))[1]
            self.blk_size_y = _first_true(int_scalars, lambda l: 'nyb' in str(l[0]))[1]
            self.blk_size_z = _first_true(int_scalars, lambda l: 'nzb' in str(l[0]))[1]

        elif self.grd_type == 'paramesh':
            self.blk_num = _first_true(int_scalars, lambda l: 'globalnumblocks' in str(l[0]))[1] 
            self.blk_num_x = _first_true(int_runtime, lambda l: 'nblockx' in str(l[0]))[1]
            self.blk_num_y = _first_true(int_runtime, lambda l: 'nblocky' in str(l[0]))[1]
            self.blk_num_z = 1
            self.blk_size_x = _first_true(int_scalars, lambda l: 'nxb' in str(l[0]))[1]
            self.blk_size_y = _first_true(int_scalars, lambda l: 'nyb' in str(l[0]))[1]
            self.blk_size_z = _first_true(int_scalars, lambda l: 'nzb' in str(l[0]))[1]

        else:
            pass # other grid handling operations

        # initialize block to grid mapping dictionaries
        self.blk_dict_x = {}
        keys: List[int] = sorted(set(self.blk_coords[:, 0]))
        for key, val in zip(keys, range(self.blk_num_x)):
            self.blk_dict_x[key] = val

        self.blk_dict_y = {}
        keys = sorted(set(self.blk_coords[:, 1]))
        for key, val in zip(keys, range(self.blk_num_y)):
            self.blk_dict_y[key] = val 
          
        self.blk_dict_z = None

        # initialize grid data
        self.grd_size_x = self.blk_num_x * self.blk_size_x
        self.grd_size_y = self.blk_num_y * self.blk_size_y
        self.grd_size_z = self.blk_num_z * self.blk_size_z
        self.grd_size = self.grd_size_x * self.grd_size_y * self.grd_size_z

    def __str__(self) -> str:
        fields = ['grd_type', 'blk_num', 'blk_num_x', 'blk_num_y', 'blk_num_z', 
                  'blk_size_x', 'blk_size_y', 'blk_size_z']
        return f'GeometryData(key={self.key:.4f}, ' + ', '.join(field + 
            '=' + str(getattr(self, field)) for field in fields) + ')'